# Deconvolução de Euler aplicada na anomalia de campo total produzida por uma esfera

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html).

### Coisas para importar

In [1]:
#import sip
#sip.setapi('QString', 2)
#sip.setapi('QVariant', 2)

In [2]:
%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import sphere
from fatiando.vis import mpl, myv

### Pontos onde são calculados os dados

In [3]:
shape = (100, 100) # número de pontos ao longo de x e y
N = shape[0]*shape[1] # número de pontos
area = [-5000, 5000, -5000, 5000] # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

### Inclinação e a declinação do campo geomagnético local

In [4]:
inc = 56. # graus
dec = -60. # graus
amp = 23500.0 # nT

In [5]:
F = utils.ang2vec(amp, inc, dec) # vetor campo geomagnético local

In [6]:
print F

[  6570.51661578 -11380.46861051  19482.38295504]


### Modelo de corpo esférico

In [7]:
model = [mesher.Sphere(0., 0., 700., 500., {'magnetization': utils.ang2vec(10., inc, dec)})]

In [8]:
scene = myv.figure(size=(800,600))
myv.points([[model[0].x, model[0].y, model[0].z]], 
           color=(1, 0, 0), size=model[0].radius, opacity=0.5)
bounds = area + [0, 3000]
ax = myv.axes(myv.outline(bounds), ranges=[b*0.001 for b in bounds], nlabels=3, fmt='%.1f')
ax.axes.x_label, ax.axes.y_label, ax.axes.z_label = 'x(km)', 'y(km)', 'z(km)'
ax.axes.font_factor = 1.2
myv.wall_bottom(bounds)
myv.wall_north(bounds)
scene.scene.camera.compute_view_plane_normal()
scene.scene.render()
myv.savefig('esfera.png')
myv.show()

C:\Anaconda\lib\site-packages\traits\has_traits.py:1766: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  setattr( self, name, value )


<img src='esfera.png', width=600>

### Anomalia de campo total (em nT)

In [15]:
act = sphere.tf(xp, yp, zp, model, inc, dec) + 30.0

### Derivadas espaciais da anomalia de campo total (em nT/m)

In [16]:
delta = 10.

dx_act = (sphere.tf(xp + delta, yp, zp, model, inc, dec) - \
          sphere.tf(xp - delta, yp, zp, model, inc, dec))/(2.*delta)

dy_act = (sphere.tf(xp, yp + delta, zp, model, inc, dec) - \
          sphere.tf(xp, yp - delta, zp, model, inc, dec))/(2.*delta)

dz_act = (sphere.tf(xp, yp, zp + delta, model, inc, dec) - \
          sphere.tf(xp, yp, zp - delta, model, inc, dec))/(2.*delta)

### Deconvolução de Euler

In [22]:
eta = 1.0

In [23]:
y = np.reshape(xp*dx_act + yp*dy_act + zp*dz_act + eta*act, (N, 1))
A = np.hstack((np.reshape(dx_act, (N, 1)), 
               np.reshape(dy_act, (N, 1)), 
               np.reshape(dz_act, (N, 1)), 
               np.zeros((N,1)) + eta))

In [24]:
x = np.linalg.solve(np.dot(A.T, A), np.dot(A.T, y))

In [25]:
print 'x: %12.5f | %12.5f' % (model[0].x,x[0])
print 'y: %12.5f | %12.5f' % (model[0].y,x[1])
print 'z: %12.5f | %12.5f' % (model[0].z,x[2])
print 'Nb: 30 | %12.5f' % x[3]

x:      0.00000 |     -0.02081
y:      0.00000 |      0.02537
z:    700.00000 |     60.08775
Nb: 30 |     24.22799


In [26]:
scene = myv.figure(size=(800,600))
myv.points(x[:3,0], size=100.)
myv.points([[model[0].x, model[0].y, model[0].z]], 
           color=(1, 0, 0), size=model[0].radius, opacity=0.5)
bounds = area + [0, 3000]
ax = myv.axes(myv.outline(bounds), ranges=[b*0.001 for b in bounds], nlabels=3, fmt='%.1f')
ax.axes.x_label, ax.axes.y_label, ax.axes.z_label = 'x(km)', 'y(km)', 'z(km)'
ax.axes.font_factor = 1.2
myv.wall_bottom(bounds)
myv.wall_north(bounds)
scene.scene.camera.compute_view_plane_normal()
scene.scene.render()
myv.savefig('esfera_estimativa.png')
myv.show()

<img src='esfera_estimativa.png', width=600>